## Import

In [1]:
import os
import random
import copy

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import wandb

In [3]:
from neumeta.models import create_mnist_model as create_model
from neumeta.utils import (
    parse_args, print_omegaconf,
    load_checkpoint, save_checkpoint,
    set_seed,
    get_cifar10, get_dataset,
    sample_coordinates, sample_subset, shuffle_coordinates_all,
    get_hypernetwork, get_optimizer, 
    sample_weights,
    weighted_regression_loss, validate_single, AverageMeter, EMA,
    sample_single_model, sample_merge_model,
)

In [4]:
from smooth.permute import PermutationManager, compute_tv_loss_for_network

## Functions

### 1 Find maximum dimension of the model

In [5]:
def find_max_dim(model_cls):
    """Find maximum dimension of the model"""
    # Get the learnable parameters of the model
    checkpoint = model_cls.learnable_parameter 

    # Set the maximum value to the length of the checkpoint
    max_value = len(checkpoint)

    # Iterate over the new model's weight
    for i, (k, tensor) in enumerate(checkpoint.items()):
        # Handle 2D tensors (e.g., weight matrices) 
        if len(tensor.shape) == 4:
            coords = [tensor.shape[0], tensor.shape[1]]
            max_value = max(max_value, max(coords))
        # Handle 1D tensors (e.g., biases)
        elif len(tensor.shape) == 1:
            max_value = max(max_value, tensor.shape[0])
    
    return max_value

### 2 Initialize wandb

In [6]:
def initialize_wandb(config):
    import time
    """
    Initializes Weights and Biases (wandb) with the given configuration.
    
    Args:
        configuration (dict): Configuration parameters for the run.
    """
    # Name the run using current time and configuration name
    run_name = f"{time.strftime('%Y%m%d%H%M%S')}-{config.experiment.name}"
    
    wandb.init(project="ninr-trial", name=run_name, config=dict(config), group='LeNet')

### 3 Initialize model dictionary

In [7]:
def init_model_dict(args, device):
    """
    Initializes a dictionary of models for each dimension in the given range, along with ground truth models for the starting dimension.

    Args:
        args: An object containing the arguments for initializing the models.

    Returns:
        dim_dict: A dictionary containing the models for each dimension, along with their corresponding coordinates, keys, indices, size, and ground truth models.
        gt_model_dict: A dictionary containing the ground truth models for the starting dimension.
    """
    dim_dict = {}
    gt_model_dict = {}
    
    # Create a model for each dimension in dimensions range
    for dim in args.dimensions.range:
        model_cls = create_model(args.model.type,
                                 hidden_dim=dim,
                                 path=args.model.pretrained_path,
                                 # smooth=args.model.smooth (Not used in LeNet)
                                 ).to(device)
        model_cls.eval()
        # Sample the coordinates, keys, indices, and the size for the model
        coords_tensor, keys_list, indices_list, size_list = sample_coordinates(model_cls)
        # Add the model, coordinates, keys, indices, size, and key mask to the dictionary
        dim_dict[f"{dim}"] = (model_cls, coords_tensor, keys_list, indices_list, size_list, None)

        # Print to makes line better
        print('\n')
        
        # If the dimension is the starting dimension (the dimension of pretrained_model), add the ground truth model to the dictionary
        if dim == args.dimensions.start:
            print(f"Loading model for dim {dim}")
            model_trained = create_model(args.model.type, 
                                         hidden_dim=dim, 
                                         path=args.model.pretrained_path, 
                                         # smooth=args.model.smooth
                                         ).to(device)
            model_trained.load_state_dict(torch.load(args.model.pretrained_path))
            model_trained.eval()

            # Smooths the ground truth model
            print("TV original model: ", compute_tv_loss_for_network(model_trained, lambda_tv=1.0).item())
            input_tensor = torch.randn(1, 1, 28, 28).to(device)
            permute_func = PermutationManager(model_trained, input_tensor)
            permute_dict = permute_func.compute_permute_dict()
            model_trained = permute_func.apply_permutations(permute_dict, ignored_keys=[
                ('conv_1.weight', 'in_channels'), 
                ('linear.weight', 'out_channels'), 
                ('linear.bias', 'out_channels')
            ])
            print("TV permutated model: ", compute_tv_loss_for_network(model_trained, lambda_tv=1.0).item())

            gt_model_dict[f'{dim}'] = copy.deepcopy(model_trained)
    
    return dim_dict, gt_model_dict

### 4 Training function for target model of a random dimension

In [8]:
# Function to train the model for one epoch
def train_one_epoch(model, train_loader, optimizer, criterion, dim_dict, gt_model_dict, epoch_idx, ema=None, args=None, device='cpu'):
    # Set the model to training mode
    model.train()
    total_loss = 0.0

    # Initialize AverageMeter objects to track the losses
    losses = AverageMeter()
    cls_losses = AverageMeter()
    reg_losses = AverageMeter()
    reconstruct_losses = AverageMeter()

    # Iterate over the training data
    for batch_idx, (x, target) in enumerate(train_loader):
        # Zero the gradients
        optimizer.zero_grad()

        # Preprocess input
        # ------------------------------------------------------------------------------------------------------
        # Move the data to the device
        x, target = x.to(device), target.to(device)
        # Choose a random hidden dimension
        hidden_dim = random.choice(args.dimensions.range)
        # Get the model class, coordinates, keys, indices, size, and key mask for the chosen dimension
        model_cls, coords_tensor, keys_list, indices_list, size_list, key_mask = dim_dict[f"{hidden_dim}"]
        # Sample a subset the input tensor of the coordinates, keys, indices, size, and selected keys
        coords_tensor, keys_list, indices_list, size_list, selected_keys = sample_subset(coords_tensor,
                                                                                         keys_list,
                                                                                         indices_list,
                                                                                         size_list,
                                                                                         key_mask,
                                                                                         ratio=args.ratio)
        # Add noise to the coordinates if specified
        if args.training.coordinate_noise > 0.0:
            coords_tensor = coords_tensor + (torch.rand_like(coords_tensor) - 0.5) * args.training.coordinate_noise


        # Main task of hypernetwork and target network
        # ------------------------------------------------------------------------------------------------------
        # Sample the weights for the target model using hypernetwork
        model_cls, reconstructed_weights = sample_weights(model, model_cls,
                                                          coords_tensor, keys_list, indices_list, size_list, key_mask, selected_keys,
                                                          device=device, NORM=args.dimensions.norm)
        # Forward pass
        predict = model_cls(x)


        # Compute losses
        # ------------------------------------------------------------------------------------------------------
        # Compute classification loss
        cls_loss = criterion(predict, target) 
        # Compute regularization loss
        reg_loss = sum([torch.norm(w, p=2) for w in reconstructed_weights])
        # Compute reconstruction loss if ground truth model is available
        if f"{hidden_dim}" in gt_model_dict:
            gt_model = gt_model_dict[f"{hidden_dim}"]
            gt_selected_weights = [
                w for k, w in gt_model.learnable_parameter.items() if k in selected_keys]

            reconstruct_loss = weighted_regression_loss(
                reconstructed_weights, gt_selected_weights)
        else:
            reconstruct_loss = torch.tensor(0.0)
        # Compute the total loss
        loss = args.hyper_model.loss_weight.ce_weight * cls_loss + args.hyper_model.loss_weight.reg_weight * \
            reg_loss + args.hyper_model.loss_weight.recon_weight * reconstruct_loss


        # Compute gradients and update weights
        # ------------------------------------------------------------------------------------------------------
        # Zero the gradients of the updated weights
        for updated_weight in model_cls.parameters():
            updated_weight.grad = None

        # Compute the gradients of the reconstructed weights
        loss.backward(retain_graph=True)
        torch.autograd.backward(reconstructed_weights, [
                                w.grad for k, w in model_cls.named_parameters() if k in selected_keys])
        
        # Clip the gradients if specified
        if args.training.get('clip_grad', 0.0) > 0:
            torch.nn.utils.clip_grad_value_(
                model.parameters(), args.training.clip_grad)
            
        # Update the weights
        optimizer.step()

        # Update the EMA if specified
        if ema:
            ema.update()  # Update the EMA after each training step
        total_loss += loss.item()

        # Update the AverageMeter objects
        losses.update(loss.item())
        cls_losses.update(cls_loss.item())
        reg_losses.update(reg_loss.item())
        reconstruct_losses.update(reconstruct_loss.item())


        # Log (or plot) losses
        # ------------------------------------------------------------------------------------------------------
        # Log the losses and learning rate to wandb
        if batch_idx % args.experiment.log_interval == 0:
            wandb.log({
                "Loss": losses.avg,
                "Cls Loss": cls_losses.avg,
                "Reg Loss": reg_losses.avg,
                "Reconstruct Loss": reconstruct_losses.avg,
                # "Recon Weight": args.hyper_model.loss_weight.recon_weight,  # Newly added for LeNet
                "Learning rate": optimizer.param_groups[0]['lr']
            }, step=batch_idx + epoch_idx * len(train_loader))
            # Print the losses and learning rate
            print(
                f"Iteration {batch_idx}: Loss = {losses.avg:.4f}, Reg Loss = {reg_losses.avg:.4f}, Reconstruct Loss = {reconstruct_losses.avg:.4f}, Cls Loss = {cls_losses.avg:.4f}, Learning rate = {optimizer.param_groups[0]['lr']:.4e}")
    
    # Returns the training loss, structure of network in each dimension, and the original structure of pretrained network
    return losses.avg, dim_dict, gt_model_dict

## Main

### 0 Set device to GPU

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### 1 Parsing arguments for inputs

In [10]:
CONFIG_PATH = 'neumeta/config/mnist/LeNet_mnist_8-32_coodnoise.yaml'
RATIO = '1.0'

In [11]:
argv_train = ['--config', CONFIG_PATH, '--ratio', RATIO]

In [12]:
args = parse_args(argv_train)  # Parse arguments
print_omegaconf(args)  # Print arguments

Loading base config from toy/experiments/base_config.yaml
+--------------------------------------+----------------------------------------------------------------------------------------------------+
|                 Key                  |                                               Value                                                |
+--------------------------------------+----------------------------------------------------------------------------------------------------+
|           experiment.name            |                                       mnist_lenet_8-32-noise                                       |
|         experiment.recononly         |                                                 0                                                  |
|        experiment.num_epochs         |                                                 50                                                 |
|       experiment.log_interval        |                                                 5

In [13]:
set_seed(args.experiment.seed)

Setting seed... 42 for reproducibility


### 2 Get training and validation data (in dataloader format)

In [14]:
train_loader, val_loader = get_dataset(args.training.dataset, args.training.batch_size, strong_transform=args.training.get('strong_aug', None))

Using dataset: mnist with batch size: 128 and strong transform: None


### 3 Create target model

#### 3.0 Create the model

In [15]:
model = create_model(args.model.type,
                     hidden_dim=args.dimensions.start,
                     path=args.model.pretrained_path,
                     # smooth=args.model.smooth
                     ).to(device)

In [16]:
# Load LeNet pretrained weight
model.load_state_dict(torch.load(args.model.pretrained_path))
model.eval()

MnistNet(
  (conv_1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=replicate)
  (conv_2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
  (conv_3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
  (f_1): ReLU()
  (f_2): ReLU()
  (f_3): ReLU()
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (linear): Linear(in_features=128, out_features=10, bias=True)
)

#### 3.1 Print the structure and shape of the model

In [17]:
model

MnistNet(
  (conv_1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=replicate)
  (conv_2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
  (conv_3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
  (f_1): ReLU()
  (f_2): ReLU()
  (f_3): ReLU()
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (linear): Linear(in_features=128, out_features=10, bias=True)
)

In [18]:
for i, (k, tensor) in enumerate(model.learnable_parameter.items()):
    print(k, tensor.shape)

conv_1.weight torch.Size([32, 1, 3, 3])
conv_1.bias torch.Size([32])
conv_2.weight torch.Size([64, 32, 5, 5])
conv_2.bias torch.Size([64])
conv_3.weight torch.Size([128, 64, 5, 5])
conv_3.bias torch.Size([128])
linear.weight torch.Size([10, 128])
linear.bias torch.Size([10])


#### 3.2 The maximum dimension of the target model

In [19]:
# Print the maximum dimension of the model
# print(f'Maximum DIM: {find_max_dim(model)}')

#### 3.3 Validate the accuracy of pretrained network

In [20]:
# Validate the model for the starting dimension (its pretrained form)
val_loss, acc = validate_single(model, val_loader, nn.CrossEntropyLoss(), args=args)
print(f'Initial Permutated model Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc * 100:.2f}%')

100%|██████████| 79/79 [00:01<00:00, 57.37it/s]

Initial Permutated model Validation Loss: 0.0621, Validation Accuracy: 98.14%


In [21]:
# Get the learnable parameters of the model
checkpoint = model.learnable_parameter
# Get the number of parameters
number_param = len(checkpoint)

In [22]:
# Print the keys of the parameters and the number of parameters
print(f"Parameters keys: {model.keys}")
print(f"Number of parameters to be learned: {number_param}")

Parameters keys: ['conv_1.weight', 'conv_1.bias', 'conv_2.weight', 'conv_2.bias', 'conv_3.weight', 'conv_3.bias', 'linear.weight', 'linear.bias']
Number of parameters to be learned: 8


### 4 Create the hypernetwork

#### 4.0 Create the model

In [23]:
# Get the hypermodel
hyper_model = get_hypernetwork(args, number_param)

Hyper model type: resmlp
Using scalar 0.1
num_freqs:  16 <class 'int'>


#### 4.1 Print model structure

In [24]:
hyper_model

NeRF_ResMLP_Compose(
  (positional_encoding): PositionalEncoding()
  (model): ModuleList(
    (0-7): 8 x NeRF_MLP_Residual_Scaled(
      (initial_layer): Linear(in_features=198, out_features=256, bias=True)
      (residual_blocks): ModuleList(
        (0-2): 3 x Linear(in_features=256, out_features=256, bias=True)
      )
      (scalars): ParameterList(
          (0): Parameter containing: [torch.float32 of size  (cuda:0)]
          (1): Parameter containing: [torch.float32 of size  (cuda:0)]
          (2): Parameter containing: [torch.float32 of size  (cuda:0)]
      )
      (act): ReLU(inplace=True)
      (output_layer): Linear(in_features=256, out_features=25, bias=True)
    )
  )
)

#### 4.2 Initialize EMA to track only a smooth version of the model weight

In [25]:
# Initialize the EMA
ema = EMA(hyper_model, decay=args.hyper_model.ema_decay)

### 5 Get Loss function, Optimizer, and Scheduler

In [26]:
criterion, val_criterion, optimizer, scheduler = get_optimizer(args, hyper_model)

In [27]:
print(f'Criterion: {criterion}\nVal_criterion: {val_criterion}\nOptimizer: {optimizer}\nScheduler: {scheduler}')

Criterion: CrossEntropyLoss()
Val_criterion: CrossEntropyLoss()
Optimizer: AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.001
    lr: 0.001
    maximize: False
    weight_decay: 0.0
)
Scheduler: <torch.optim.lr_scheduler.MultiStepLR object at 0x0000021C47E4FFD0>


### 6 Training loop

#### 6.1 Initialize training parameters

In [28]:
# Initialize the starting epoch and best accuracy
start_epoch = 0
best_acc = 0.0

#### 6.2 Directory to save the model

In [29]:
# Create the directory to save the model
os.makedirs(args.training.save_model_path, exist_ok=True)

#### 6.3 Resume training loop

In [30]:
if args.resume_from:
        print(f"Resuming from checkpoint: {args.resume_from}")
        checkpoint_info = load_checkpoint(args.resume_from, hyper_model, optimizer, ema)
        start_epoch = checkpoint_info['epoch']
        best_acc = checkpoint_info['best_acc']
        print(f"Resuming from epoch: {start_epoch}, best accuracy: {best_acc*100:.2f}%")
        # Note: If there are more elements to retrieve, do so here.

#### 6.4 Initialize wandb for plotting

In [31]:
# Initialize wandb
initialize_wandb(args)

wandb: Currently logged in as: efradosuryadi (efradosuryadi-universitas-indonesia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


#### 6.5 Initialize model dictionary for each dimension and shuffle it

In [32]:
# Initialize model dictionary
dim_dict, gt_model_dict = init_model_dict(args, device)



















































Loading model for dim 32
TV original model:  3088.543212890625
TV permutated model:  2175.4375


In [33]:
gt_model_dict

{'32': MnistNet(
   (conv_1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=replicate)
   (conv_2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
   (conv_3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
   (f_1): ReLU()
   (f_2): ReLU()
   (f_3): ReLU()
   (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
   (linear): Linear(in_features=128, out_features=10, bias=True)
 )}

In [34]:
dim_dict

{'8': (MnistNet(
    (conv_1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=replicate)
    (conv_2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
    (conv_3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
    (f_1): ReLU()
    (f_2): ReLU()
    (f_3): ReLU()
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (linear): Linear(in_features=32, out_features=10, bias=True)
  ),
  tensor([[ 0.,  0.,  0.,  8.,  8.,  1.],
          [ 0.,  1.,  0.,  8.,  8.,  1.],
          [ 0.,  2.,  0.,  8.,  8.,  1.],
          ...,
          [ 7.,  7.,  0.,  8., 10.,  1.],
          [ 7.,  8.,  0.,  8., 10.,  1.],
          [ 7.,  9.,  0.,  8., 10.,  1.]]),
  array(['conv_1.weight', 'conv_1.weight', 'conv_1.weight', ...,
         'linear.bias', 'linear.bias', 'linear.bias'], dtype='<U13'),
  tensor([[0, 0, 3, 3],
          [1, 0, 3, 3],
          [2, 0, 3, 3],
          ...,
 

In [35]:
dim_dict = shuffle_coordinates_all(dim_dict)
dim_dict

{'8': (MnistNet(
    (conv_1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=replicate)
    (conv_2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
    (conv_3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=replicate)
    (f_1): ReLU()
    (f_2): ReLU()
    (f_3): ReLU()
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (linear): Linear(in_features=32, out_features=10, bias=True)
  ),
  tensor([[ 0.,  0.,  0.,  8.,  8.,  1.],
          [ 0.,  1.,  0.,  8.,  8.,  1.],
          [ 0.,  2.,  0.,  8.,  8.,  1.],
          ...,
          [ 7.,  7.,  0.,  8., 10.,  1.],
          [ 7.,  8.,  0.,  8., 10.,  1.],
          [ 7.,  9.,  0.,  8., 10.,  1.]]),
  array(['conv_1.weight', 'conv_1.weight', 'conv_1.weight', ...,
         'linear.bias', 'linear.bias', 'linear.bias'], dtype='<U13'),
  tensor([[0, 0, 3, 3],
          [1, 0, 3, 3],
          [2, 0, 3, 3],
          ...,
 

#### 6.6 Hypernetwork training loop

In [36]:
args.experiment.num_epochs

50

In [37]:
# Iterate over the epochs
for epoch in range(start_epoch, args.experiment.num_epochs):
    train_loss, dim_dict, gt_model_dict = train_one_epoch(hyper_model, train_loader, optimizer, criterion, 
                                                          dim_dict, gt_model_dict, epoch_idx=epoch, ema=ema, 
                                                          args=args, device=device)
    # Step the scheduler
    scheduler.step()

    # Print the training loss and learning rate
    print(f"Epoch [{epoch+1}/{args.experiment.num_epochs}], Training Loss: {train_loss:.4f}, Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

    # If it's time to evaluate the model
    if (epoch + 1) % args.experiment.eval_interval == 0:
        # Apply EMA if it is specified
        if ema:
            ema.apply()  # Save the weights of original model created before training_loop
        
        # Sample the merged model (create model of same structure before training loop by using the hypernetwork)
        # And then test the performance of the hypernetwork by seeing how good it is in generating the weights
        model = sample_merge_model(hyper_model, model, args) 
        # Validate the merged model
        val_loss, acc = validate_single(model, val_loader, val_criterion, args=args)

        # If EMA is specified, restore the original weights
        if ema:
            ema.restore()  # Restore the original weights to the weights of the pretrained networks

        # Log the validation loss and accuracy to wandb
        wandb.log({
            "Validation Loss": val_loss,
            "Validation Accuracy": acc
        })
        # Print the validation loss and accuracy
        print(f"Epoch [{epoch+1}/{args.experiment.num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc*100:.2f}%")
        print('\n\n')

        # Save the checkpoint if the accuracy is better than the previous best
        if acc > best_acc:
            best_acc = acc
            save_checkpoint(f"{args.training.save_model_path}/mnist_nerf_best.pth",hyper_model,optimizer,ema,epoch,best_acc)
            print(f"Checkpoint saved at epoch {epoch} with accuracy: {best_acc*100:.2f}%")


Iteration 0: Loss = 2.3828, Reg Loss = 11.6718, Reconstruct Loss = 0.0000, Cls Loss = 2.3817, Learning rate = 1.0000e-03
Iteration 50: Loss = 2.3194, Reg Loss = 10.1162, Reconstruct Loss = 0.0010, Cls Loss = 2.3174, Learning rate = 1.0000e-03
Iteration 100: Loss = 2.3180, Reg Loss = 9.9594, Reconstruct Loss = 0.0007, Cls Loss = 2.3163, Learning rate = 1.0000e-03
Iteration 150: Loss = 2.3146, Reg Loss = 9.3854, Reconstruct Loss = 0.0007, Cls Loss = 2.3129, Learning rate = 1.0000e-03
Iteration 200: Loss = 2.3007, Reg Loss = 14.8758, Reconstruct Loss = 0.0029, Cls Loss = 2.2963, Learning rate = 1.0000e-03
Iteration 250: Loss = 2.2684, Reg Loss = 25.5748, Reconstruct Loss = 0.0035, Cls Loss = 2.2623, Learning rate = 1.0000e-03
Iteration 300: Loss = 2.2431, Reg Loss = 34.4771, Reconstruct Loss = 0.0029, Cls Loss = 2.2368, Learning rate = 1.0000e-03
Iteration 350: Loss = 2.2250, Reg Loss = 40.6304, Reconstruct Loss = 0.0032, Cls Loss = 2.2177, Learning rate = 1.0000e-03
Iteration 400: Loss =

100%|██████████| 79/79 [00:01<00:00, 54.10it/s]


Epoch [1/50], Validation Loss: 2.1636, Validation Accuracy: 20.85%



Checkpoint saved at epoch 0 with accuracy: 20.85%
Iteration 0: Loss = 1.9897, Reg Loss = 84.2579, Reconstruct Loss = 0.0000, Cls Loss = 1.9812, Learning rate = 1.0000e-03
Iteration 50: Loss = 2.1158, Reg Loss = 85.4009, Reconstruct Loss = 0.0069, Cls Loss = 2.1004, Learning rate = 1.0000e-03
Iteration 100: Loss = 2.1027, Reg Loss = 82.5665, Reconstruct Loss = 0.0057, Cls Loss = 2.0888, Learning rate = 1.0000e-03
Iteration 150: Loss = 2.0919, Reg Loss = 82.5984, Reconstruct Loss = 0.0038, Cls Loss = 2.0799, Learning rate = 1.0000e-03
Iteration 200: Loss = 2.0891, Reg Loss = 81.0112, Reconstruct Loss = 0.0063, Cls Loss = 2.0747, Learning rate = 1.0000e-03
Iteration 250: Loss = 2.0877, Reg Loss = 78.5290, Reconstruct Loss = 0.0074, Cls Loss = 2.0725, Learning rate = 1.0000e-03
Iteration 300: Loss = 2.0844, Reg Loss = 76.2417, Reconstruct Loss = 0.0071, Cls Loss = 2.0697, Learning rate = 1.0000e-03
Iteration 350: Loss = 

100%|██████████| 79/79 [00:01<00:00, 55.98it/s]


Epoch [2/50], Validation Loss: 2.0498, Validation Accuracy: 19.61%



Iteration 0: Loss = 2.1258, Reg Loss = 71.7127, Reconstruct Loss = 0.0000, Cls Loss = 2.1186, Learning rate = 1.0000e-03
Iteration 50: Loss = 2.1568, Reg Loss = 73.8484, Reconstruct Loss = 0.0077, Cls Loss = 2.1417, Learning rate = 1.0000e-03
Iteration 100: Loss = 2.1069, Reg Loss = 75.2805, Reconstruct Loss = 0.0077, Cls Loss = 2.0916, Learning rate = 1.0000e-03
Iteration 150: Loss = 2.0954, Reg Loss = 74.3859, Reconstruct Loss = 0.0085, Cls Loss = 2.0795, Learning rate = 1.0000e-03
Iteration 200: Loss = 2.0860, Reg Loss = 73.2251, Reconstruct Loss = 0.0064, Cls Loss = 2.0723, Learning rate = 1.0000e-03
Iteration 250: Loss = 2.0816, Reg Loss = 72.1324, Reconstruct Loss = 0.0081, Cls Loss = 2.0662, Learning rate = 1.0000e-03
Iteration 300: Loss = 2.0751, Reg Loss = 70.8331, Reconstruct Loss = 0.0077, Cls Loss = 2.0603, Learning rate = 1.0000e-03
Iteration 350: Loss = 2.0681, Reg Loss = 70.2187, Reconstruct Loss = 0.0

...

Epoch [48/50], Validation Loss: 0.1579, Validation Accuracy: 95.23%



Checkpoint saved at epoch 47 with accuracy: 95.23%
Iteration 0: Loss = 0.2193, Reg Loss = 69.5850, Reconstruct Loss = 0.0000, Cls Loss = 0.2124, Learning rate = 1.0000e-03
Iteration 50: Loss = 0.2356, Reg Loss = 70.1508, Reconstruct Loss = 0.0026, Cls Loss = 0.2260, Learning rate = 1.0000e-03
Iteration 100: Loss = 0.2369, Reg Loss = 70.4525, Reconstruct Loss = 0.0033, Cls Loss = 0.2266, Learning rate = 1.0000e-03
Iteration 150: Loss = 0.2312, Reg Loss = 70.6285, Reconstruct Loss = 0.0035, Cls Loss = 0.2206, Learning rate = 1.0000e-03
Iteration 200: Loss = 0.2254, Reg Loss = 70.5150, Reconstruct Loss = 0.0030, Cls Loss = 0.2154, Learning rate = 1.0000e-03
Iteration 250: Loss = 0.2252, Reg Loss = 70.3911, Reconstruct Loss = 0.0029, Cls Loss = 0.2153, Learning rate = 1.0000e-03
Iteration 300: Loss = 0.2233, Reg Loss = 70.4327, Reconstruct Loss = 0.0031, Cls Loss = 0.2132, Learning rate = 1.0000e-03
Iteration 350: Loss 

100%|██████████| 79/79 [00:01<00:00, 53.10it/s]


Epoch [49/50], Validation Loss: 0.1551, Validation Accuracy: 95.29%



Checkpoint saved at epoch 48 with accuracy: 95.29%
Iteration 0: Loss = 0.1248, Reg Loss = 67.5277, Reconstruct Loss = 0.0000, Cls Loss = 0.1180, Learning rate = 1.0000e-03
Iteration 50: Loss = 0.2144, Reg Loss = 69.6593, Reconstruct Loss = 0.0012, Cls Loss = 0.2062, Learning rate = 1.0000e-03
Iteration 100: Loss = 0.2141, Reg Loss = 69.2963, Reconstruct Loss = 0.0018, Cls Loss = 0.2054, Learning rate = 1.0000e-03
Iteration 150: Loss = 0.2151, Reg Loss = 69.5883, Reconstruct Loss = 0.0024, Cls Loss = 0.2057, Learning rate = 1.0000e-03
Iteration 200: Loss = 0.2142, Reg Loss = 69.5703, Reconstruct Loss = 0.0028, Cls Loss = 0.2044, Learning rate = 1.0000e-03
Iteration 250: Loss = 0.2126, Reg Loss = 69.6513, Reconstruct Loss = 0.0025, Cls Loss = 0.2031, Learning rate = 1.0000e-03
Iteration 300: Loss = 0.2110, Reg Loss = 69.5766, Reconstruct Loss = 0.0031, Cls Loss = 0.2010, Learning rate = 1.0000e-03
Iteration 350: Loss 

100%|██████████| 79/79 [00:01<00:00, 55.60it/s]

Epoch [50/50], Validation Loss: 0.1527, Validation Accuracy: 95.09%





In [42]:
# Validate the model for the starting dimension (its pretrained form)
val_loss, acc = validate_single(dim_dict['8'][0], val_loader, nn.CrossEntropyLoss(), args=args)
print(f'Initial Permutated model Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc * 100:.2f}%')

100%|██████████| 79/79 [00:01<00:00, 51.09it/s]

Initial Permutated model Validation Loss: 0.2343, Validation Accuracy: 92.52%


In [43]:
# Validate the model for the starting dimension (its pretrained form)
val_loss, acc = validate_single(dim_dict['32'][0], val_loader, nn.CrossEntropyLoss(), args=args)
print(f'Initial Permutated model Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc * 100:.2f}%')

100%|██████████| 79/79 [00:01<00:00, 46.00it/s]

Initial Permutated model Validation Loss: 0.1670, Validation Accuracy: 95.00%


In [44]:
# Validate the model for the starting dimension (its pretrained form)
val_loss, acc = validate_single(gt_model_dict['32'], val_loader, nn.CrossEntropyLoss(), args=args)
print(f'Initial Permutated model Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc * 100:.2f}%')

100%|██████████| 79/79 [00:01<00:00, 47.00it/s]

Initial Permutated model Validation Loss: 0.0621, Validation Accuracy: 98.14%


In [45]:
# End the wandb tracking
wandb.finish()

Cls Loss,██▇▇▇▇▆▆▆▅▄▄▄▄▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁
Learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss,█▇▇▇█▇▇▇▆▆▅▅▄▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Reconstruct Loss,▁▂▃▃▁▃▄▄▄▃▄▁█▂▅▆▇▄▄▃▁▄▃▄▃▃▃▂▃▃▃▃▃▃▃▂▁▂▂▂
Reg Loss,▁█▇████████▇▇▇▇▇▇▇▇▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
Validation Accuracy,▁▁▁▂▂▂▃▄▄▄▅▅▆▆▆▇▇▇██████████████████████
Validation Loss,███▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Cls Loss,0.1986
Learning rate,0.001
Loss,0.20854
Reconstruct Loss,0.00297


### 7 Testing loop

In [48]:
saved_hypernet_path = args.training.save_model_path + 'mnist_nerf_best.pth'

In [49]:
saved_hypernet_path

'toy/experiments/mnist_lenet_8-32-noise/mnist_nerf_best.pth'

In [50]:
hyper_model_test = get_hypernetwork(args, number_param)

Hyper model type: resmlp
Using scalar 0.1
num_freqs:  16 <class 'int'>


In [51]:
checkpoint = torch.load(saved_hypernet_path, map_location="cpu")  # or "cuda" if using GPU
hyper_model_test.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [52]:
for hidden_dim in range(4, 48):
    # Create a model for this given dimension
    model = create_model(args.model.type,
                         hidden_dim=hidden_dim,
                         path=args.model.pretrained_path,
                         # smooth=args.model.smooth
                         ).to(device)
    
    # If EMA is specified, apply it
    if ema:
        print('Applying EMA')
        ema.apply()

    # Sample the merged model
    accumulated_model = sample_merge_model(hyper_model_test, model, args, K=100)

    # Validate the merged model
    val_loss, acc = validate_single(accumulated_model, val_loader, val_criterion, args=args)

    # If EMA is specified, restore the original weights after applying EMA
    if ema:
        ema.restore()  # Restore the original weights after applying 
        
    # Save the model
    save_name = os.path.join(args.training.save_model_path, f"mnist_{accumulated_model.__class__.__name__}_dim{hidden_dim}_single.pth")
    torch.save(accumulated_model.state_dict(),save_name)

    # Print the results
    print(f"Test using model {args.model}: hidden_dim {hidden_dim}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc*100:.2f}%")
    print('\n')

    # Define the directory and filename structure
    filename = f"mnist_results_{args.experiment.name}.txt"
    filepath = os.path.join(args.training.save_model_path, filename)

    # Write the results. 'a' is used to append the results; a new file will be created if it doesn't exist.
    with open(filepath, "a") as file:
        file.write(f"Hidden_dim: {hidden_dim}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {acc*100:.2f}%\n")


Applying EMA


100%|██████████| 79/79 [00:01<00:00, 68.04it/s]


Test using model {'type': 'LeNet', 'pretrained_path': 'toy/mnist_MnistNet_dim32.pth', 'smooth': False}: hidden_dim 4, Validation Loss: 1.4290, Validation Accuracy: 60.85%


Applying EMA


100%|██████████| 79/79 [00:01<00:00, 68.50it/s]


Test using model {'type': 'LeNet', 'pretrained_path': 'toy/mnist_MnistNet_dim32.pth', 'smooth': False}: hidden_dim 5, Validation Loss: 1.0547, Validation Accuracy: 70.98%


Applying EMA


...

Test using model {'type': 'LeNet', 'pretrained_path': 'toy/mnist_MnistNet_dim32.pth', 'smooth': False}: hidden_dim 45, Validation Loss: 0.1774, Validation Accuracy: 94.93%


Applying EMA


100%|██████████| 79/79 [00:02<00:00, 38.85it/s]


Test using model {'type': 'LeNet', 'pretrained_path': 'toy/mnist_MnistNet_dim32.pth', 'smooth': False}: hidden_dim 46, Validation Loss: 0.1801, Validation Accuracy: 95.00%


Applying EMA


100%|██████████| 79/79 [00:01<00:00, 40.79it/s]

Test using model {'type': 'LeNet', 'pretrained_path': 'toy/mnist_MnistNet_dim32.pth', 'smooth': False}: hidden_dim 47, Validation Loss: 0.1736, Validation Accuracy: 94.89%


